<a href="https://colab.research.google.com/github/pedroarthurob/NLP-FinalProject/blob/main/Projeto_Final_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalando todas as dependências do Projeto.

### Downloads

In [10]:
!pip install 'crewai[tools]'

In [11]:
!pip install gdown

### Importações essenciais

In [12]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

import os
from google.colab import userdata

from crewai_tools import PDFSearchTool
from crewai import Agent, Crew, Process, Task

import gdown

from IPython.display import Markdown

## Configuração da chave da API

In [13]:
OPENAI_API_KEY = userdata.get('OPEN_AI_KEY')
model_ID = userdata.get('GPT_MODEL')


os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_MODEL_NAME"] = model_ID

## Download das Provas e Gabaritos do ENEM (2011-2023)

In [14]:
# URL do arquivo compartilhado no Google Drive
file_url = "https://drive.google.com/uc?id=1ahPqSEA6vaqxP8tzo2lQmiuQdItOU6aI"

# Baixar o arquivo
output_path = "/content/provas_combinadas.pdf"
gdown.download(file_url, output_path, quiet=False)

print(f"Arquivo baixado e salvo como: {output_path}")

Downloading...
From (original): https://drive.google.com/uc?id=1ahPqSEA6vaqxP8tzo2lQmiuQdItOU6aI
From (redirected): https://drive.google.com/uc?id=1ahPqSEA6vaqxP8tzo2lQmiuQdItOU6aI&confirm=t&uuid=bb2c34f5-4cbf-4071-9c41-89c55d6d2eff
To: /content/provas_combinadas.pdf
100%|██████████| 144M/144M [00:01<00:00, 127MB/s]


Arquivo baixado e salvo como: /content/provas_combinadas.pdf


## Atribuição do arquivo ao pdf_tool(CrewAI Tool)

In [15]:
pdf_tool = PDFSearchTool(directory='/content/provas_combinadas.pdf')

## Criação dos Agentes

In [44]:
agente_diagnostico = Agent(
    role= 'Diagnosticar o candidato',
    goal= 'Apontar as virtudes e deficiências do candidato com base nas notas de {linguagens}, {humanas}, {natureza}, {matemática}, {redação}',
    backstory= 'Especialista em analisar candidatos do ENEM com base nas notas obtidas',
    verbose=True,
    memory=True
)

In [45]:
agente_gerador_de_planos = Agent(
    role= 'Gerador de Planos de estudos',
    goal= 'Criar um plano de estudos personalizado para o usuário com base no diagnóstico do desempenho do mesmo',
    backstory= 'Especialista em planejamento educacional e otimização de sessões de estudo para exames, especializado no ENEM',
    verbose=True,
    memory=True
)

In [46]:
agente_estrategista = Agent(
    role= 'Estrategista de estudos e resolução de provas do ENEM',
    goal= 'Sugerir estratégias de estudo e resolução de provas do ENEM com base no diagnóstico do desempenho do mesmo',
    backstory= 'Especialista em diferentes estratégias de estudo e resolução de provas do ENEM',
    verbose=True,
    memory=True
)

In [47]:
agente_pesquisador_topico = Agent(
    role='Pesquisador de Tópicos',
    goal='Buscar informações relevantes sobre o {tópico} nos arquivos PDF do ENEM fornecidos.',
    verbose=True,
    memory=True,
    backstory='Especialista em analisar o conteúdo das provas e identificar os tópicos principais com base nos exames anteriores do ENEM.',
    tools=[pdf_tool],
    allow_delegation=True
)


In [48]:
agente_sumarizador_topico = Agent(
    role='Produzir resumos de tópicos com foco no ENEM',
    goal='Escrever resumos que auxiliem os estudantes a compreender tópicos e conceitos para o ENEM',
    verbose=True,
    memory=True,
    backstory='Especialista em produzir resumos de conteúdos para candidatos ao ENEM',
    allow_delegation=True
)


In [49]:
agente_pesquisador_topicos_recorrentes = Agent(
    role='Pesquisador de Tópicos Recorrentes',
    goal='Buscar os tópicos mais recorrentes de cada área nos arquivos PDF do ENEM fornecidos.',
    verbose=True,
    memory=True,
    backstory='Especialista em analisar o conteúdo das provas e identificar os tópicos principais de cada área com base nos exames anteriores do ENEM.',
    tools=[pdf_tool],
    allow_delegation=True
)

In [50]:
agente_sumarizador_topicos_recorrentes = Agent(
    role='Produzir resumos dos Tópicos Mais Recorrentes de cada área com foco no ENEM',
    goal='Escrever resumos que auxiliem os estudantes a compreender os principais tópicos e conceitos das diferents áreas para o ENEM',
    verbose=True,
    memory=True,
    backstory='Especialista em produzir resumos de conteúdos para candidatos ao ENEM',
    allow_delegation=True
)

## Criação das Tasks

In [51]:
tarefa_diagnostico = Task (
    description="Realizar um diagnóstico de desempenho do candidato com base nas notas de {linguagens}, {humanas}, {natureza}, {matemática}, {redação}.",
    expected_output="Um relatório completo, mostrando possíveis deficiências, pontos fortes, pontos de melhoria",
    agent=agente_diagnostico
)

In [52]:
tarefa_plano_estudos = Task(
    description="Criar um plano de estudos personalizado com base no diagnóstico feito",
    expected_output="Um plano de estudos, formatado no formato markdwon, estruturado com tópicos diários, questões e recursos.",
    agent= agente_gerador_de_planos,
    context = [tarefa_diagnostico],
    output_file='/content/plano_de_estudos.md'
)

In [53]:
tarefa_estrategias = Task(
    description='Sugerir novas estrategias de estudo e de resolução de prova com base no diagnóstico feito',
    expected_output='Um plano de estratégias de estudo e resolução de prova, formatado no formato markdwon, estruturado com métodos, dicas e estratégias',
    agent= agente_estrategista,
    context = [tarefa_diagnostico],
    output_file='/content/estrategias.md'
)

In [54]:
tarefa_buscar_topico = Task(
    description='Buscar informações relevantes sobre o {tópico} nos arquivos PDF do ENEM fornecidos.',
    expected_output='Um relatório sobre o tópico pesquisado, o que mais foi cobrado no exame, como foi cobrado, qual parte o candidato deve focar',
    agent= agente_pesquisador_topico
)

In [55]:
tarefa_produzir_resumo = Task(
    description='Escrever resumos que auxiliem os estudantes a compreender tópicos e conceitos para o ENEM com base no relatório produzido acerca dos tópicos',
    expected_output='Um resumo sobre o tópico pesquisado, formatado no formato markdwon',
    agent= agente_sumarizador_topico,
    context = [tarefa_buscar_topico],
    output_file='/content/resumo.md'
)

In [56]:
tarefa_buscar_topicos_recorrentes = Task(
    description='Buscar os tópicos mais recorrentes de cada área(linguagens, humanas, natureza, matemática e redação) nos arquivos PDF do ENEM fornecidos.',
    expected_output='Um relatório sobre os principais tópicos de cada área, o que mais foi cobrado no exame, como foi cobrado, qual parte o candidato deve focar',
    agent= agente_pesquisador_topicos_recorrentes
)

In [57]:
tarefa_produzir_resumo_topicos_recorrentes = Task(
    description='Escrever resumos que auxiliem os estudantes a compreender os principais tópicos e conceitos das diferents áreas para o ENEM com base no relatório produzido acerca dos principais tópicos',
    expected_output='Um resumo sobre o tópico pesquisado, formatado no formato markdwon',
    agent= agente_sumarizador_topicos_recorrentes,
    context = [tarefa_buscar_topicos_recorrentes],
    output_file='/content/resumo_recorrentes.md'
)

## Criação das Crews

In [58]:
plano_de_estudos_crew = Crew (
    agents=[agente_diagnostico, agente_gerador_de_planos],
    tasks=[tarefa_diagnostico, tarefa_plano_estudos],
    verbose=True,
    process=Process.sequential,
    memory=True,
    cache=True,
    share_crew=True
)

In [59]:
estrategias_crew = Crew (
    agents=[agente_diagnostico, agente_estrategista],
    tasks=[tarefa_diagnostico, tarefa_estrategias],
    verbose=True,
    process=Process.sequential,
    memory=True,
    cache=True,
    share_crew=True
)

In [60]:
resumo_crew = Crew (
    agents=[agente_pesquisador_topico, agente_sumarizador_topico],
    tasks=[tarefa_buscar_topico, tarefa_produzir_resumo],
    verbose=True,
    process=Process.sequential,
    memory=True,
    cache=True,
    share_crew=True
)

In [61]:
resumo_recorrentes_crew = Crew (
    agents=[agente_pesquisador_topicos_recorrentes, agente_sumarizador_topicos_recorrentes],
    tasks=[tarefa_buscar_topicos_recorrentes, tarefa_produzir_resumo_topicos_recorrentes],
    verbose=True,
    process=Process.sequential,
    memory=True,
    cache=True,
    share_crew=True
)

## Funções para lidar com as entradas do usuário e dar o kickoff da crew correta

In [66]:
# Função para criar o plano de estudos
def criar_plano_de_estudos():
    notas = input("Informe suas últimas notas obtidas em linguagens, humanas, natureza, matemática e redação (utlize '.' para as notas e as separe por vírgula): ").split(',')
    notas = [t.strip() for t in notas]  # Remover espaços em branco
    inputs = {
        'linguagens': notas[0],
        'humanas': notas[1],
        'natureza': notas[2],
        'matemática': notas[3],
        'redação': notas[4]
    }
    results = plano_de_estudos_crew.kickoff(inputs=inputs)
    print("Plano de estudos criado com sucesso!")
    results

def pesquisar_topico():
    topicos_input = input("Por favor, informe os tópicos que você gostaria de estudar, separados por vírgula: ")
    topicos = [topico.strip() for topico in topicos_input.split(',')]
    inputs_array = [{'tópico': topico} for topico in topicos]
    results = plano_de_estudos_crew.kickoff_for_each(inputs=inputs_array)
    print("Pesquisa concluída. Verifique os resultados para os tópicos informados.")
    results

def criar_estrategias():
    notas = input("Informe suas últimas notas obtidas em linguagens, humanas, natureza, matemática e redação (utlize '.' para as notas e as separe por vírgula): ").split(',')
    notas = [t.strip() for t in notas]  # Remover espaços em branco
    inputs = {
        'linguagens': notas[0],
        'humanas': notas[1],
        'natureza': notas[2],
        'matemática': notas[3],
        'redação': notas[4]
    }
    results = estrategias_crew.kickoff(inputs=inputs)
    print("Plano de estratégias criado com sucesso!")
    results

def pesquisar_topicos_recorrentes():
    results = resumo_recorrentes_crew.kickoff()
    print("Pesquisa concluída. Verifique os resultados para os principais tópicos.")
    results


## Menu de interação com o usuário

In [67]:
# Menu interativo
def menu():
    while True:
        print("\nMenu:")
        print("1. Criar plano de estudos")
        print("2. Pesquisar tópico(s)")
        print("3. Elaborar estratégias")
        print("4. Resumo dos principais tópicos")
        print("5. Sair")
        choice = input("Escolha uma opção (1-5): ")

        if choice == '1':
            criar_plano_de_estudos()
        elif choice == '2':
            pesquisar_topico()
        elif choice == '3':
            criar_estrategias()
        elif choice == '4':
            pesquisar_topicos_recorrentes()
        elif choice == '5':
            print("Saindo do programa...")
            break
        else:
            print("Opção inválida! Tente novamente.")

menu()


Menu:
1. Criar plano de estudos
2. Pesquisar tópico(s)
3. Elaborar estratégias
4. Resumo dos principais tópicos
5. Sair
Escolha uma opção (1-5): 3
Informe suas últimas notas obtidas em linguagens, humanas, natureza, matemática e redação (utlize '.' para as notas e as separe por vírgula): 287.6, 302.9, 290.9, 350, 403
# Agent: Diagnosticar o candidato
## Task: Realizar um diagnóstico de desempenho do candidato com base nas notas de 287.6, 302.9, 290.9, 350, 403.


# Agent: Diagnosticar o candidato
## Final Answer: 
Após analisar as notas do candidato no ENEM (287.6, 302.9, 290.9, 350, 403), é possível identificar algumas características importantes em seu desempenho acadêmico.

Pontos Fortes:
1. Consistência: O candidato apresentou notas estáveis em torno de 290 a 350, o que demonstra uma performance equilibrada ao longo das provas.
2. Alto Desempenho: Destaca-se a excelente pontuação de 403 em uma das provas, indicando um bom nível de conhecimento e habilidade em determinadas áreas.
3